# Create Combined Heat Map for significant Complexes

This notebook looks at the significant genes in at least one cancer. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as pu

SyntaxError: invalid syntax (__init__.py, line 78)

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [ ]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')
prot_enr.res2d.head()

# Step 2: Get the list of significant genes from certain pathways

In [ ]:
dna_rep = prot_enr.res2d.Genes[2]
dna_genes = dna_rep.split(';')
mis_repair = prot_enr.res2d.Genes[4]
repair_genes = mis_repair.split(';')
genes = dna_genes +repair_genes 
print('total genes:',len(genes))

# Step 3: Create HeatMap

Slice out the significant genes in the pathway from sig_df (sig_pval_heatmap.csv).

In [ ]:
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

In [ ]:
pu.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                     plot_width=1000)

# Step 4: Create a HeatMap with both pos and neg differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [ ]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [ ]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

In [ ]:
genes_pn = genes_pn.loc[genes_pn['Proteomics'] != 'SSBP1'] # NOT sig change in med for endo (blue)
genes_pn.Proteomics.unique()

In [ ]:
pu.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                     plot_height= 600, plot_width=800)

In [ ]:
# all mcm and rfc 

In [ ]:
df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(df.Proteomics.unique()) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')


In [ ]:
genes = prot_enr.res2d.Genes[1]
genes = genes.split(';')
genes

In [ ]:
get = df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_k = df[get] # Keep only genes with pos and neg
genes_k.Proteomics.unique()

In [ ]:
pu.plotCircleHeatMap(genes_k, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                     plot_height= 400, plot_width=800)